In [3]:
from onnxruntime.quantization import quantize_dynamic, QuantType
import onnx
from onnx import numpy_helper
import os 

In [15]:

model_fp32 = '/home/airi/yolo/ONNX-TensorRT-Pytorch-Tensorflow-Face-Detection-Models-Quantization/models/yolov8l.onnx'
model_quant = '/home/airi/yolo/ONNX-TensorRT-Pytorch-Tensorflow-Face-Detection-Models-Quantization/quant_models/quant_yolov8l.onnx'


In [4]:
def convert_bytes(num):
    """
    this function will convert bytes to MB.... GB... etc
    """
    for x in ['bytes', 'KB', 'MB', 'GB', 'TB']:
        if num < 1024.0:
            return "%3.1f %s" % (num, x)
        num /= 1024.0


def file_size(file_path):
    """
    this function will return the file size
    """
    if os.path.isfile(file_path):
        file_info = os.stat(file_path)
        return convert_bytes(file_info.st_size)

In [5]:
# Lets check the file size of MS Paint exe 
# or you can use any file path
file_path = r"/home/airi/yolo/ONNX-TensorRT-Pytorch-Tensorflow-Face-Detection-Models-Quantization/models/yolov8l.onnx"
print(file_size(file_path))

166.6 MB


In [6]:
# Quantized model
file_path = r"/home/airi/yolo/ONNX-TensorRT-Pytorch-Tensorflow-Face-Detection-Models-Quantization/quant_models/quant_yolov8l.onnx"
print(file_size(file_path))

42.0 MB


In [7]:
# Model Float16 Conversion
file_path = r"/home/airi/yolo/ONNX-TensorRT-Pytorch-Tensorflow-Face-Detection-Models-Quantization/quant_models/model_fp16.onnx"
print(file_size(file_path))

83.4 MB


In [3]:
model = onnx.load(model_fp32)
INTIALIZERS=model.graph.initializer
Weight=[]
for initializer in INTIALIZERS:
    W= numpy_helper.to_array(initializer)
    Weight.append(W)
print(f"Layers: {len(Weight)}")
print(Weight[0][0][0])
print(f"Type: {Weight[0][0][0][0][0].dtype}")


Layers: 207
[[-0.00551987]]
Type: float32


### Dynamic quantization

In [4]:
quantize_dynamic(model_fp32,  model_quant, weight_type=QuantType.QUInt8) #chnage QInt8 to QUInt8

In [5]:

import onnxruntime as ort
ort_session = ort.InferenceSession(model_quant, providers=['CPUExecutionProvider'])

In [6]:
model  = onnx.load(model_quant)
INTIALIZERS=model.graph.initializer
Weight=[]
for initializer in INTIALIZERS:
    W= numpy_helper.to_array(initializer)
    Weight.append(W)
print(f"Layers: {len(Weight)}")
print(Weight[0][0])
print(f"Type: {Weight[0][0].dtype}")

Layers: 567
2.9414062
Type: float32


### Float16 Conversion
   
   Converting a model to use float16 instead of float32 can decrease the model size (up to half) and improve performance 
on some GPUs. There may be some accuracy loss, but in many models the new accuracy is acceptable. Tuning data is not needed for float16 conversion, which can make it preferable to quantization.

In [17]:
from onnxconverter_common import float16

model_fp32 = '/home/airi/yolo/ONNX-TensorRT-Pytorch-Tensorflow-Face-Detection-Models-Quantization/models/yolov8l.onnx'
model = onnx.load(model_fp32)
model_fp16 = float16.convert_float_to_float16(model)
onnx.save(model_fp16, "/home/airi/yolo/ONNX-TensorRT-Pytorch-Tensorflow-Face-Detection-Models-Quantization/quant_models/model_fp16.onnx")

/home/airi/miniconda3/envs/mahmud/lib/python3.9/site-packages/onnxconverter_common/float16.py:43: UserWarning: the float32 number 7.522191936004674e-08 will be truncated to 1e-07
  warnings.warn("the float32 number {} will be truncated to {}".format(pos_min, min_positive_val))
/home/airi/miniconda3/envs/mahmud/lib/python3.9/site-packages/onnxconverter_common/float16.py:53: UserWarning: the float32 number -6.257487683569707e-08 will be truncated to -1e-07
  warnings.warn("the float32 number {} will be truncated to {}".format(neg_max, -min_positive_val))
/home/airi/miniconda3/envs/mahmud/lib/python3.9/site-packages/onnxconverter_common/float16.py:43: UserWarning: the float32 number 9.744358919760998e-08 will be truncated to 1e-07
  warnings.warn("the float32 number {} will be truncated to {}".format(pos_min, min_positive_val))
/home/airi/miniconda3/envs/mahmud/lib/python3.9/site-packages/onnxconverter_common/float16.py:53: UserWarning: the float32 number -9.744358919760998e-08 will be tr

In [19]:
model  = onnx.load("/home/airi/yolo/ONNX-TensorRT-Pytorch-Tensorflow-Face-Detection-Models-Quantization/quant_models/model_fp16.onnx")
INTIALIZERS=model.graph.initializer
Weight=[]
for initializer in INTIALIZERS:
    W= numpy_helper.to_array(initializer)
    Weight.append(W)
print(f"Layers: {len(Weight)}")
print(Weight[0][0][0])
print(f"Type: {Weight[0][0].dtype}")

Layers: 207
[[-0.00552]]
Type: float16
